count, list and order the frequency of words essential
count, list and order the frequency of keywords essential
allow user to select reference corpus essential
allow user to select statistical formular for keyness
display the first 20 words/keywords of each dataset

order the known datasets by number of shared words/keywords with questionned dataset
display the shared words/keywords in the first 20 words/keywords of each dataset essential
identify the most similiar dataset based on the highest number of shared top 20 words/keywords of the questionned dataset
identify the least similiar dataset based on the lowest number of shared top 20 words/keywords of the questionned dataset essential
suggest to rule out the least similiar dataset essential
allow user to confirm or reject ruling out
continue to identify the most/least similiar dataset and rule out the least for the top 40, then 60 keywords

tag the part-of-speech (POS) of each word essential
allow the user to select any word or string and display the word or string in context essential
show 6 words before the target and 6 after the target word for each instance of the word in each dataset
allow the user to search for POS patterns following the target word, e.g. absolutely + JJ essential
count the number of identical POS patterns in each dataset essential
order the known datasets by number of identical POS patterns with the questionned dataset
identify the most similiar dataset based on the number of identical POS patterns with questionned dataset
identify the least similiar dataset based on the number of identical POS patterns with questionned dataset essential
suggest ruling out the least similiar dataset essential
allow user to confirm or reject ruling out

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("preprocessed_enron.csv")
df['body']

for i, msg in enumerate(df['body']):
    # print(i, msg)

    if msg is np.nan:
        df = df.drop(i)


for i, msg in enumerate(df['body']):
    if msg is np.nan:
        print(i, msg)

df = df.reset_index(drop=True)


In [ ]:
len(df)

2854

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
documents = df['body'].tolist()

# for i, msg in enumerate(df['body']):
#     # print(i, msg)
#     if msg is np.nan:
#         print(i, msg)

#tfidf_vectorizer = TfidfVectorizer()
#tfidf_vectors = tfidf_vectorizer.fit_transform(documents) # keyword frequency list
tf_vectorizer = CountVectorizer()
tf_vectors = tf_vectorizer.fit_transform(documents)         # word frequency list
# del tf_vectorizer
# del tfidf_vectorizer# データ分割r


In [ ]:
print(documents)

In [ ]:
# 作成された辞書を作る　:トレインデータ・テストデータ両方に対応
words=tf_vectorizer.get_feature_names_out()
print(words[5020:5025])

['kitchen' 'kkenyan' 'kll' 'klussmann' 'km']


In [ ]:
#tfidf_mat = tfidf_vectors.toarray() # dead every time
#del tfidf_vectors
tf_mat = tf_vectors.toarray()
del tf_vectors
df['tf'] = tf_mat.tolist()
#df['tfidf'] = tfidf_mat.tolist()

In [ ]:
# 0 ベクトルを消去 Normalization のため
for i, vec in enumerate(df['tf']):
    if sum(vec) == 0:
        df = df.drop(i)

df = df.reset_index(drop=True)

In [ ]:
# Normalization

# we generaly name 'ntf' for normalized term frequency
normalized_tf_list = []
for row in df['tf']:
    num_words = sum(row)
    normalized_tf = []
    for x in row:
        normalized_tf.append(x/num_words)

    normalized_tf_list.append(normalized_tf)

df['ntf'] = normalized_tf_list

In [ ]:
import math
math.log2(0)

ValueError: ignored

#Tf-idf の代わりに利用する keyness を作る

ここでは　df['keyness'] を作成し追加したい

In [ ]:
import math
# we generaly name 'ntf' for normalized term frequency


# First, create the shared normalized tf vector
shared_ntf = None # shared ntf of all document

matrix = []
for row in df['ntf']:
    matrix.append(row)

np_matrix = np.array(matrix)

mean_vector = np_matrix.mean(axis=0)

shared_ntf = mean_vector.tolist()



def keyness(ntf_vector1, ref_ntf_vector2): # freq_vector1 and freq_vector2 are both already normalized
    keyness_vec = []
    for i, x in enumerate(ntf_vector1):
        if ntf_vector1[i] == 0:
            keyness_vec.append(0)
        else:
            keyness_vec.append(math.log2(ntf_vector1[i]/ref_ntf_vector2[i]))

    return keyness_vec


keyness_mat = []
for ntf_vector in df['ntf']:
    keyness_vec = keyness(ntf_vector, shared_ntf)
    keyness_mat.append(keyness_vec)



In [ ]:
df['keyness'] = keyness_mat
df

,author,body,tf,ntf,keyness
0,chris.stokley@enron.com,Please view the summary tab on each worksheet ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,chris.stokley@enron.com,Please view the summary tab on each worksheet ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,chris.stokley@enron.com,Please view the summary tab on each worksheet ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,chris.stokley@enron.com,Please stop by this morning for cake and cooki...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,chris.stokley@enron.com,Please stop by this morning for cake and cooki...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...
2649,ben.jacoby@enron.com,I talked to Bill Butler after this came out. B...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2650,ben.jacoby@enron.com,"Gentlemen:\n\nFurther to our conversation, ple...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.02040816326530612, 0.0, 0.0, 0.0, 0.0, 0.0,...","[3.8738423573420007, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2651,ben.jacoby@enron.com,I talked to Bill Butler after this came out. B...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2652,ben.jacoby@enron.com,Charles:\n\nIt was good to catch up with you t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
i = 22
msg = df['body'][i]
max_value = max(keyness_mat[i])
max_idx = keyness_mat[i].index(max_value)
print(words[max_idx])

print(msg)
#print(df['author'][i])


mechelle
Mechelle,
            I will let you make the call, MW's or $'s. Look at the spreadsheet  below and give me your thoughts.
 
                                                              Chris
              




# データ分割

In [ ]:
from sklearn.model_selection import train_test_split
X_tfidf_train, X_tfidf_test, Y_tfidf_train, Y_tfidf_test = train_test_split(df['tfidf'],df['author'],test_size=0.2,shuffle=True)
X_tf_train, X_tf_test, Y_tf_train, Y_tf_test = train_test_split(df['tf'],df['author'],test_size=0.2,shuffle=True)

In [ ]:
# How many author?
authors = set(Y_tfidf_test)
authors_list = [author for author in authors]

# Feature_vectors の作成
size: 著者の数

Train データから作る

In [ ]:
# df_X = pd.DataFrame(X_tf_train.values.tolist())
# df_Y = pd.DataFrame(Y_tf_train.values.tolist())
df_concat = pd.concat((X_tfidf_train, Y_tfidf_train.rename('author')), axis=1)

reference_vectors = {}
for author in authors:

    df_author = df_concat.groupby('author').get_group(author)

    matrix = []
    for row in df_author['tfidf']:
        matrix.append(row)

    np_matrix = np.array(matrix)

    mean_vector = np_matrix.mean(axis=0)
    reference_vectors[author] = mean_vector.tolist()

In [ ]:
authors

{'andy.zipper@enron.com',
 'ben.jacoby@enron.com',
 'chris.stokley@enron.com',
 'hunter.shively@enron.com',
 'j..kean@enron.com',
 'v.weldon@enron.com'}

# Problem

Tf-idf でも　the や　to などの一般的な単語が強調されてしまった。emails のデータセットの場合だと、全ての文章に含まれているわけではないので抑制作用が弱くなる

# Solution

nltk の　stop_words から commonly used words を　取得し、それらを省いた中でのtop20 を出す

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
authors

{'andy.zipper@enron.com',
 'ben.jacoby@enron.com',
 'chris.stokley@enron.com',
 'hunter.shively@enron.com',
 'j..kean@enron.com',
 'v.weldon@enron.com'}

In [ ]:
print(stop_words)
len(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

179

In [ ]:
# start からend までのwordの配列を返す
def extract_features_words(freq_vector, words,stop_words, start=0, end=20):

    setX = set(freq_vector) # 最大値を取り出すため set を作成

    count = 0

    result = []

    while count<end:
        max_value = max(setX)
        max_index = freq_vector.index(max_value)
        max_word = words[max_index]

        setX.remove(max_value)

        if max_word not in stop_words:
            if count>= start:
                result.append(max_word)
            count+=1
        # count += 1


    return result

In [ ]:
# start からend までのword IDの配列を返す
def extract_features(freq_vector, words,stop_words, start=0, end=20):

    setX = set(freq_vector) # 最大値を取り出すため set を作成

    count = 0

    result = []

    while count<end:
        max_value = max(setX)
        max_index = freq_vector.index(max_value)
        max_word = words[max_index]

        setX.remove(max_value)

        if max_word not in stop_words:
            if count>= start:
                result.append(max_index)
            count+=1
        # count += 1


    return result

In [ ]:
vec = reference_vectors['v.weldon@enron.com']
top_words = extract_features_words(vec, words=words, stop_words=stop_words,  start=0, end=150)
print(top_words)


['charlie', 'thanks', 'com', 'http', 'enron', '20', 'weldon', 'know', 'good', 'www', 'let', 'asp', 'gas', 'one', 'got', 'thank', 'need', 'sure', 'would', 'lunch', 'link', 'call', 'ect', 'fuel', 'data', 'time', 'go', 'may', 'get', 'likely', '2001', '7229', '01', 'year', 'going', 'corp', 'weather', 'want', 'sounds', 'per', 'framework', 'bought', 'dev', '2000', 'based', '345', 'default', 'think', 'stagecoach', 'today', 'charles', 'html', 'last', 'info', 'could', 'new', 'news', 'previous', 'come', '713', 'research', 'pretty', 'energy', 'note', 'saw', 'melissa', 'next', 'still', 'updated', 'yahoo', 'looking', 'gov', 'bid', 'money', 'curves', 'htm', '22', 'ene', 'gasfundy', 'work', 'check', 'vanadium', 'great', 'capacity', 'subject', 'please', 'much', 'attend', 'see', 'really', 'less', 'week', 'oil', 'years', 'wanted', 'like', 'niagara', 'prices', 'buy', 'show', 'yesterday', 'sell', 'also', 'company', 'able', 'hours', 'doug', 'night', 'cold', 'cc', 'business', 'tonight', 'hurricane', 'assist

# ReferenceVectors 完成

## next

# predict() 関数の作成

In [ ]:
def get_similarity(feature_vector1,feature_vector2):
    return len(set(feature_vector1) & set(feature_vector2))



In [ ]:

def predict(questioned_vector,reference_vectors):
    start = 0
    end = 20
    suspected = [author for author in authors]
    similarityWithQ = {}
    while(len(suspected) > 1):
        Q_features = extract_features(questioned_vector, words,start, end)
        for author, reference_vector in reference_vectors.items():
            feature_vector = extract_features(reference_vector, words, start, end)
            score = get_similarity(reference_vector,Q_features)
            similarityWithQ[author]=score
        print(f'{min(similarityWithQ, key=similarityWithQ.get)} is minimum')

        end += 20


In [ ]:
# def predict(questioned_vector, reference_vectors):
#     suspected = [author for author in authors]

#     comparedSize = 20
#     while(len(suspected) > 1):






SyntaxError: ignored